# Machine Learning Model for Therapist Ratings

This notebook is used to build and evaluate a machine learning model to predict therapist ratings based on various factors such as review sentiment, location, and specialties.

In [1]:
# Import necessary libraries
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator


In [3]:

# Initialize Spark session
spark = SparkSession.builder.appName("TherapistRatingPrediction").getOrCreate()

# Load processed data
data_path = "../data/processed/therapists_data.parquet"
df = spark.read.parquet(data_path)

# Display the schema of the data
df.printSchema()

# Data preprocessing
indexer = StringIndexer(inputCol="specialty", outputCol="specialtyIndex")
model = indexer.fit(df)
df = model.transform(df)

# Assemble features
feature_columns = ["location", "specialtyIndex", "review_sentiment"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df)

# Split the data into training and test sets
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

# Train the model
lr = LinearRegression(featuresCol="features", labelCol="rating")
lr_model = lr.fit(train_data)

# Evaluate the model
test_results = lr_model.evaluate(test_data)
print(f"RMSE: {test_results.rootMeanSquaredError}")
print(f"R2: {test_results.r2}")

# Save the model
model_path = "../models/therapist_rating_model"
lr_model.save(model_path)

# Stop the Spark session
spark.stop()

root
 |-- Name: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Profile_URL: string (nullable = true)
 |-- Statement: string (nullable = true)
 |-- Phone: string (nullable = true)



Py4JJavaError: An error occurred while calling o44.fit.
: org.apache.spark.SparkException: Input column specialty does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115)
	at org.apache.spark.